In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf

# **데이터 로드**<br>
Seq2Seq를 이용한 한글 챗봇을 만들기 위해서<br>
송영숙님이 공개한 한글 데이터셋을 사용합니다.<br>
https://github.com/songys/Chatbot_data<br>
또한 다음을 참고하여 만들었습니다<br>
참고자료:https://wikidocs.net/86900

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData%20.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
question=list(train_data['Q'])
answer=list(train_data['A'])
for i in range(5):
  print('Q: '+question[i])
  print('A: '+answer[i])
  print()

Q: 12시 땡!
A: 하루가 또 가네요.

Q: 1지망 학교 떨어졌어
A: 위로해 드립니다.

Q: 3박4일 놀러가고 싶다
A: 여행은 언제나 좋죠.

Q: 3박4일 정도 놀러가고 싶다
A: 여행은 언제나 좋죠.

Q: PPL 심하네
A: 눈살이 찌푸려지죠.



데이터 전처리

In [ ]:
pip install konlpy

In [5]:
import konlpy.tag
from konlpy.tag import Okt

In [6]:
def preprocessing(sentences):

  okt=konlpy.tag.Okt()
  #KoNLPy 형태소분석기 설정
  
  after_preprocess=[]

  for sentence in sentences:
    sentence=re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》;]', '', sentence)
    #구두점 제거

    #okt.morphs(sentence)
    #12시 땡->['12시', '땡']
    #1지망 학교 떨어졌어->['1', '지망', '학교', '떨어졌어']
    after_preprocess.append(" ".join(okt.morphs(sentence)))
  return after_preprocess

In [7]:
question=preprocessing(question)
answer=preprocessing(answer)

for i in range(5):
  print('Q: ', end='')
  print(question[i])
  print('A: ', end='')
  print(answer[i])
  print()

Q: 12시 땡
A: 하루 가 또 가네요

Q: 1 지망 학교 떨어졌어
A: 위로 해 드립니다

Q: 3 박 4일 놀러 가고 싶다
A: 여행 은 언제나 좋죠

Q: 3 박 4일 정도 놀러 가고 싶다
A: 여행 은 언제나 좋죠

Q: PPL 심하네
A: 눈살 이 찌푸려지죠



Seq2Seq에서는 학습시 총 3개의 데이터가 필요합니다.<br><br>
인코더 입력: 12시 땡<br>
디코더 입력: sos 하루 가 또 가네요<br>
디코더 출력: 하루 가 또 가네요 eos<br><br>
학습시에는 이와 같은 데이터를 필요로 합니다

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
x_encoder=[]
x_decoder=[]
y_decoder=[]
for i in range(len(answer)):
  x_encoder.append(question[i].split())
  x_decoder.append(("<sos> "+answer[i]).split())
  y_decoder.append((answer[i]+" <eos>").split())
print(x_encoder[:5])
print(x_decoder[:5])
print(y_decoder[:5])

[['12시', '땡'], ['1', '지망', '학교', '떨어졌어'], ['3', '박', '4일', '놀러', '가고', '싶다'], ['3', '박', '4일', '정도', '놀러', '가고', '싶다'], ['PPL', '심하네']]
[['<sos>', '하루', '가', '또', '가네요'], ['<sos>', '위로', '해', '드립니다'], ['<sos>', '여행', '은', '언제나', '좋죠'], ['<sos>', '여행', '은', '언제나', '좋죠'], ['<sos>', '눈살', '이', '찌푸려지죠']]
[['하루', '가', '또', '가네요', '<eos>'], ['위로', '해', '드립니다', '<eos>'], ['여행', '은', '언제나', '좋죠', '<eos>'], ['여행', '은', '언제나', '좋죠', '<eos>'], ['눈살', '이', '찌푸려지죠', '<eos>']]


자연어처리에서 단어 토큰 위주로 한다면 정수 인코딩을 해야 임베딩 레이어에 넣을 수 있기 때문에 정수 인코딩을 위한 질문, 답변을 합친 전체 단어 사전을 만듭니다. 이때, 질문, 답변을 합치는 것은 같은 언어이기 때문이고 만약 번역기를 만든다면 따로 만들어줍니다

In [12]:
total_list=x_encoder+x_decoder+y_decoder

In [13]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(total_list)
vocab_size=len(tokenizer.word_index)+1
print(vocab_size)
print(tokenizer.word_index)
#형태소 분석된 결과를 바탕으로 전체 단어 사전 만들기

12650
{'<sos>': 1, '<eos>': 2, '이': 3, '을': 4, '거': 5, '가': 6, '예요': 7, '사람': 8, '요': 9, '에': 10, '도': 11, '은': 12, '해보세요': 13, '를': 14, '사랑': 15, '것': 16, '생각': 17, '수': 18, '잘': 19, '마음': 20, '의': 21, '보세요': 22, '봐요': 23, '더': 24, '는': 25, '하는': 26, '안': 27, '할': 28, '말': 29, '하고': 30, '나': 31, '너무': 32, '게': 33, '좋은': 34, '연락': 35, '이별': 36, '해': 37, '시간': 38, '많이': 39, '같아요': 40, '한': 41, '하세요': 42, '있어요': 43, '저': 44, '있을': 45, '좋아하는': 46, '때': 47, '만': 48, '마세요': 49, '내': 50, '으로': 51, '하지': 52, '들': 53, '이에요': 54, '못': 55, '친구': 56, '좀': 57, '그': 58, '건': 59, '에서': 60, '자신': 61, '일': 62, '죠': 63, '한테': 64, '썸': 65, '다': 66, '당신': 67, '고': 68, '에게': 69, '하면': 70, '적': 71, '로': 72, '이제': 73, '수도': 74, '오늘': 75, '다른': 76, '해요': 77, '네': 78, '있는': 79, '가봐요': 80, '제': 81, '좋을': 82, '같아': 83, '싶어': 84, '걸': 85, '인': 86, '랑': 87, '정리': 88, '지금': 89, '될': 90, '연애': 91, '뭐': 92, '혼자': 93, '이네': 94, '일이': 95, '해주세요': 96, '그런': 97, '남자': 98, '돼요': 99, '없어요': 100, '해도': 101, '바랄게요': 102, '같

In [14]:
encoder_input=tokenizer.texts_to_sequences(x_encoder)
decoder_input=tokenizer.texts_to_sequences(x_decoder)
decoder_output=tokenizer.texts_to_sequences(y_decoder)
print(encoder_input[0])
print(decoder_input[0])
print(decoder_output[0])
#정수 인코딩 진행

[5964, 8817]
[1, 209, 6, 123, 2467]
[209, 6, 123, 2467, 2]


In [15]:
encoder_input = pad_sequences(encoder_input, maxlen=30, padding="post")
decoder_input = pad_sequences(decoder_input, maxlen=30, padding="post")
decoder_output = pad_sequences(decoder_output, maxlen=30, padding="post")
print(encoder_input[0])
print(decoder_input[0])
print(decoder_output[0])
#패딩 진행

[5964 8817    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[   1  209    6  123 2467    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[ 209    6  123 2467    2    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


훈련을 마치고 예측을 할 때 변환하기 위한 단어 집합을 생성합니다

In [16]:
src_to_index=tokenizer.word_index
index_to_src=tokenizer.index_word
print(src_to_index)
print(index_to_src)

{'<sos>': 1, '<eos>': 2, '이': 3, '을': 4, '거': 5, '가': 6, '예요': 7, '사람': 8, '요': 9, '에': 10, '도': 11, '은': 12, '해보세요': 13, '를': 14, '사랑': 15, '것': 16, '생각': 17, '수': 18, '잘': 19, '마음': 20, '의': 21, '보세요': 22, '봐요': 23, '더': 24, '는': 25, '하는': 26, '안': 27, '할': 28, '말': 29, '하고': 30, '나': 31, '너무': 32, '게': 33, '좋은': 34, '연락': 35, '이별': 36, '해': 37, '시간': 38, '많이': 39, '같아요': 40, '한': 41, '하세요': 42, '있어요': 43, '저': 44, '있을': 45, '좋아하는': 46, '때': 47, '만': 48, '마세요': 49, '내': 50, '으로': 51, '하지': 52, '들': 53, '이에요': 54, '못': 55, '친구': 56, '좀': 57, '그': 58, '건': 59, '에서': 60, '자신': 61, '일': 62, '죠': 63, '한테': 64, '썸': 65, '다': 66, '당신': 67, '고': 68, '에게': 69, '하면': 70, '적': 71, '로': 72, '이제': 73, '수도': 74, '오늘': 75, '다른': 76, '해요': 77, '네': 78, '있는': 79, '가봐요': 80, '제': 81, '좋을': 82, '같아': 83, '싶어': 84, '걸': 85, '인': 86, '랑': 87, '정리': 88, '지금': 89, '될': 90, '연애': 91, '뭐': 92, '혼자': 93, '이네': 94, '일이': 95, '해주세요': 96, '그런': 97, '남자': 98, '돼요': 99, '없어요': 100, '해도': 101, '바랄게요': 102, '같이': 10

훈련 모델을 만듭니다. 이전에는 전부 Sequential 방식의 모델이였지만 Seq2Seq의 경우에는 인코더와 디코더가 따로 분리되어 있기 때문에 함수형 API 모델을 사용해야 합니다.

따라서, Model() 함수를 통해서 인코더와 디코더를 따로 만듭니다.

In [17]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

임베딩 벡터와 LSTM의 히든 레이더 차원을 각각 지정합니다.

In [18]:
embedding_dim=100
lstm_hidden_dim=128
#단어 개수는 vocab_size에 저장되어 있음

In [19]:
#훈련 모델 인코더 정의


encoder_inputs=Input(shape=(None,))
encoder_emb=Embedding(vocab_size, embedding_dim)(encoder_inputs)
#인코더 임베딩 층
encoder_masking=Masking(mask_value=0.0)(encoder_emb)
#패딩 0은 연산에서 제외
encoder_lstm=LSTM(lstm_hidden_dim,
                  dropout=0.1,
                  recurrent_dropout=0.5,
                  return_state=True
                  )
encoder_outputs, state_h, state_c=encoder_lstm(encoder_masking)
#상태값 리턴을 위해 return_state는 True
#LSTM은 hidden state(은닉 상태)를 위한 state_h, cell state(셀 상태)를 위한 state_c 2개의 상태 존재

encoder_states=[state_h, state_c]
#이것이 context vector가 되며 디코더의 초기 입력으로 들어가는 값

In [20]:
#훈련 모델 디코더 정의

decoder_inputs=Input(shape=(None,))
decoder_emb_layer=Embedding(vocab_size, embedding_dim)# 임베딩 층
decoder_emb=decoder_emb_layer(decoder_inputs)
#디코더 임베딩 층
decoder_masking=Masking(mask_value=0.0)(decoder_emb)
#패딩 0은 연산에서 제외
decoder_lstm=LSTM(lstm_hidden_dim,
                         dropout=0.1,
                         recurrent_dropout=0.5,
                         return_state=True,
                         return_sequences=True)
#인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
#모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_outputs, _, _=decoder_lstm(decoder_masking,
                                   initial_state=encoder_states)
#initial_state를 인코더의 상태(context vector)로 초기화
#디코더는 이미 학습된 모델을 재사용하여 새로운 디코더 예측 모델을 만들어야 하기 때문에 두개로 분리

decoder_dense=Dense(vocab_size, activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)
#단어의 개수만큼 노드의 개수로 설정하여 softmax로써 설정

In [21]:
#입력과 출력을 정의하여 함수형 API 모델 생성
model=Model([encoder_inputs, decoder_inputs], decoder_outputs)

Seq2Seq의 디코더는 기본적으로 다중분류 문제를 푸는 것으로 softmax를 사용하는데 이때, categorical_crossentropy를 사용했습니다.<br><br>
그런데 이는 레이블이 원-핫 인코딩이 된 상태여야 합니다. 하지만 지금은 그런 상태가 아니기 때문에 이 경우는 sparse_categorical_crossentropy를 사용하게 됩니다.

In [22]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.fit([encoder_input, decoder_input],
          decoder_output,
          epochs=100,
          batch_size=64)

Epoch 1/100
185/185 [==============================] - 63s 300ms/step - loss: 3.2082 - accuracy: 0.7663
Epoch 2/100
185/185 [==============================] - 56s 302ms/step - loss: 1.2485 - accuracy: 0.8222
Epoch 3/100
185/185 [==============================] - 57s 307ms/step - loss: 1.1754 - accuracy: 0.8270
Epoch 4/100
185/185 [==============================] - 57s 306ms/step - loss: 1.1314 - accuracy: 0.8317
Epoch 5/100
185/185 [==============================] - 56s 305ms/step - loss: 1.1038 - accuracy: 0.8348
Epoch 6/100
185/185 [==============================] - 57s 306ms/step - loss: 1.0690 - accuracy: 0.8396
Epoch 7/100
185/185 [==============================] - 56s 302ms/step - loss: 1.0593 - accuracy: 0.8410
Epoch 8/100
185/185 [==============================] - 57s 306ms/step - loss: 1.0138 - accuracy: 0.8452
Epoch 9/100
185/185 [==============================] - 56s 305ms/step - loss: 0.9919 - accuracy: 0.8468
Epoch 10/100
185/185 [==============================] - 55s 296m

이번에는 예측용 디코더 모델을 따로 정의합니다.<br><br> 디코더 모델의 경우 훈련용과 예측용이 다릅니다. 훈련용 디코더 모델의 경우, 모든 시점에 대해서 이전 시점의 실제값을 넣어주는 교사 강요를 진행하지만 예측용 디코더 모델의 경우 이전 시점의 예측값을 넣어주는 방식이기 때문에 따로 만들어 주게 됩니다.<br><br>
또한, 예측 모델은 이미 학습된 훈련 모델의 레이어들을 그대로 재사용하고 예측 모델 인코더는 훈련 모델 인코더와 동일합니다.

In [24]:
#예측 모델 디코더 정의

#예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
#매번 이전 디코더 상태를 입력으로 받아서 새로 설정
#이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h=Input(shape=(lstm_hidden_dim,))
decoder_state_input_c=Input(shape=(lstm_hidden_dim,))
decoder_states_inputs=[decoder_state_input_h, decoder_state_input_c]

decoder_predict_emb=decoder_emb_layer(decoder_inputs)
#임베딩 레이어

decoder_outputs2, state_h2, state_c2=decoder_lstm(decoder_predict_emb, initial_state=decoder_states_inputs)

#예측용은 이전 시점이 initial_state
decoder_states2=[state_h2, state_c2]
decoder_outputs2=decoder_dense(decoder_outputs2)

In [25]:
#예측 모델 인코더 정의
encoder_model = Model(encoder_inputs, encoder_states)

In [26]:
decoder_model=Model(
    [decoder_inputs]+decoder_states_inputs,
    [decoder_outputs2]+decoder_states2)

훈련 모델 학습

In [27]:
def generate(input_seq):
  tmp=input_seq
  input_seq=[]
  input_seq.append(tmp)
  input_seq=preprocessing(input_seq)
  input_seq=tokenizer.texts_to_sequences(input_seq)
  input_seq = pad_sequences(input_seq, maxlen=30, padding="post")
  states_value=encoder_model.predict(input_seq)
  #context vector 얻음
  target_seq=np.zeros((1, 1))
  target_seq[0, 0]=src_to_index['<sos>']
  #예측 시 디코더에 들어가는 첫번째 입력은 <sos>
  #<sos>이후부터 <eos>가 나올때까지 예측을 계속하는 방식
  stop_condition=False
  decoded_sentence=''
  while not stop_condition:
    output_tokens, h, c=decoder_model.predict([target_seq]+states_value)
    #가장 처음에는 <sos>와 인코더의 출력인 context vector가 들어감
    #이후에는 target_seq는 계속 예측한 단어, states_value는 디코더의 셀 상태가 들어감
    predicted_token_index=np.argmax(output_tokens[0, -1, :])
    predicted_letter=index_to_src[predicted_token_index]
    #softmax 이기 때문에 가장 높은 확률인 것의 index를 받고 단어로 변환
    # print(predicted_letter)
    if predicted_letter!='<eos>':
      decoded_sentence+=' '+predicted_letter

    if (predicted_letter=='<eos>' or len(list(decoded_sentence.split()))>50):
      stop_condition=True
    
    target_seq=np.zeros((1,1))
    target_seq[0, 0]=predicted_token_index
    #다음 예측된 단어의 정수 인코딩 값

    states_value=[h, c]
    #디코더 LSTM의 이전 상태값이 다음의 입력값으로 들어가도록
  return decoded_sentence

In [28]:
print(generate('1지망 학교 떨어졌어'))

 위로 해 드립니다


In [34]:
print(generate('여행 가고 싶어'))

 즐거운 시간 보내고 오세요


질의 응답 데이터가 훨씬 많다면 더 정확하고 다양한 답변을 받을 수 있게 됩니다